In [14]:
import pandas as pd
import numpy as np
import json

df = pd.read_excel('Copied Statements.xlsx')

question = {}

for index, row in df.iterrows():
    if not pd.isna(row['Questions']):
        current_question = int(row['Questions'])
        question[current_question] = []
    question[current_question].append(row['Statements'].strip('\"'))

live_data = json.dumps(question, indent=4)

In [19]:
with open('assessment_google_sheet.json', 'r') as f:
    data = json.load(f)

model_data = {}
for question_number, statements in data.items():
    model_data[question_number] = [statement["Decision Statement"] for statement in statements]

model_data = json.dumps(model_data, indent=4)
print(model_data)


{
    "1": [
        "I rely on deep experience to understand potential customer challenges.",
        "I validate my grasp of customer challenges by discussing problems and solutions with a small group.",
        "I use quick surveys or discussions with a larger group of potential customers to validate challenges.",
        "I actively seek to understand and respond to customer challenges, validating my solution with them.",
        "I actively explore my customers' challenges through in-depth discussions without presenting my solution."
    ],
    "2": [
        "I usually handle conflict of ideas by myself to favor progress.",
        "I often prioritize my ideas during conflicts but need a convincing argument to change my approach.",
        "I occasionally consider input from others and try to reach common ground when conflict arises.",
        "I actively engage in collaborative discussions and seek common ground in idea conflicts.",
        "I value diverse opinions and encourag

In [26]:
import pandas as pd
import json

# Read the Excel file
df = pd.read_excel('Copied Statements.xlsx')

# Initialize an empty dictionary for the questions and their statements
live_data_dict = {}

# Iterate through the DataFrame
for index, row in df.iterrows():
    if not pd.isna(row['Questions']):
        current_question = int(row['Questions'])
        live_data_dict[current_question] = []
    live_data_dict[current_question].append(row['Statements'].strip('\"'))

# Convert the live data to a JSON string
live_data = json.dumps(live_data_dict, indent=4)

# Read the model data from the JSON file
with open('assessment_google_sheet.json', 'r') as f:
    model_data_dict = json.load(f)

# Initialize a dictionary to hold the discrepancies
discrepancies = []

# Compare the live_data with the model_data
for question_number, live_statements in live_data_dict.items():
    model_statements = [statement["Decision Statement"] for statement in model_data_dict.get(str(question_number), [])]
    
    missing_in_model = [statement for statement in live_statements if statement not in model_statements]
    extra_in_model = [statement for statement in model_statements if statement not in live_statements]

    if missing_in_model or extra_in_model:
        discrepancies.append({
            "Question Number": question_number,
            "Missing in Model": "; ".join(missing_in_model),
            "Extra in Model": "; ".join(extra_in_model)
        })

# Convert the discrepancies to a DataFrame
discrepancies_df = pd.DataFrame(discrepancies)

# Save the discrepancies to a CSV file
discrepancies_df.to_csv('discrepancies.csv', index=False)

# Save the discrepancies to a JSON file
with open('discrepancies.json', 'w') as json_file:
    json.dump(discrepancies, json_file, indent=4)


discrepancies_df


,Question Number,Missing in Model,Extra in Model
0,3,I take extra time to gain clarity before decid...,I take whatever time is required to gain clari...
1,4,I'm open to learning new approaches for produc...,I am open to learning new approaches for produ...
2,8,I prefer making decisions independently withou...,I engage both close contacts and experts in de...
3,28,I prefer not to spend time educating potential...,I prefer not to spend time educating potential...
4,29,I often rely on intuition and personal experie...,"I often use intuition and personal experience,..."
5,35,"I prioritize proactive problem-solving, identi...","I prioritize proactive problem-solving, identi..."
6,40,I see the initial focus on sustainable models ...,I see initial focus on sustainable models impr...


In [28]:
import pandas as pd
import json

# Read the Excel file
df = pd.read_excel('Copied Statements.xlsx')

# Initialize an empty dictionary for the questions and their statements
live_data_dict = {}

# Iterate through the DataFrame
for index, row in df.iterrows():
    if not pd.isna(row['Questions']):
        current_question = int(row['Questions'])
        live_data_dict[current_question] = []
    live_data_dict[current_question].append(row['Statements'].strip('\"'))

# Convert the live data to a JSON string
live_data = json.dumps(live_data_dict, indent=4)

# Read the model data from the JSON file
with open('assessment_google_sheet.json', 'r') as f:
    model_data_dict = json.load(f)

# Initialize a list to hold the comparison results
comparison_results = []

# Compare the live_data with the model_data considering the order
for question_number, live_statements in live_data_dict.items():
    model_statements = [statement["Decision Statement"] for statement in model_data_dict.get(str(question_number), [])]

    max_len = max(len(live_statements), len(model_statements))
    
    for i in range(max_len):
        live_statement = live_statements[i] if i < len(live_statements) else None
        model_statement = model_statements[i] if i < len(model_statements) else None
        match = (live_statement == model_statement)
        
        comparison_results.append({
            "Question Number": question_number,
            "Statement in Live Data": live_statement,
            "Statement in Model Data": model_statement,
            "Match": match
        })

# Convert the comparison results to a DataFrame
comparison_df = pd.DataFrame(comparison_results)

# Save the comparison results to a CSV file
comparison_df.to_csv('comparison_results.csv', index=False)

# Print the comparison results
comparison_df


,Question Number,Statement in Live Data,Statement in Model Data,Match
0,1,I rely on deep experience to understand potent...,I rely on deep experience to understand potent...,True
1,1,I validate my grasp of customer challenges by ...,I validate my grasp of customer challenges by ...,True
2,1,I use quick surveys or discussions with a larg...,I use quick surveys or discussions with a larg...,True
3,1,I actively seek to understand and respond to c...,I actively seek to understand and respond to c...,True
4,1,I actively explore my customers' challenges th...,I actively explore my customers' challenges th...,True
...,...,...,...,...
220,45,I have avoided dealing with urgent issues when...,I have avoided dealing with urgent issues when...,True
221,45,I seek support for urgent issues when necessar...,I seek support for urgent issues when necessar...,True
222,45,I usually prioritize and seek solutions for ur...,I usually prioritize and seek solutions for ur...,True
223,45,I consistently prioritize and address urgent i...,I consistently prioritize and address urgent i...,True
